# Import

In [1]:
import pandas as pd
import numpy as np
import sqlite3

# Neo4J Queries

    - provider_nodes
        - organization_nodes
        - taxonomy_nodes
    - hospital_nodes
        - organization_nodes
        - taxonomy_nodes
    - referral_relationships

In [2]:
db = sqlite3.connect('../data/hopteam.sqlite')

## provider_nodes
    - entity type 1
    - any cbsa
    - referred to an entity type 2 in cbsa 34980
        - with at least 50 transactions and
        - with less than 50 avg wait time

In [3]:
%%time
# referrals from providers (entity 1) to metro nashville groups (entity 2) 
# having over 50 transactions and under 50 average wait time
query = """
        
        WITH metro_entity_2 AS (
            SELECT *
            FROM profile
            INNER JOIN taxonomy
            ON profile.taxonomy_code = taxonomy.code
            WHERE provider_business_practice_location_address_postal_code IN (
                SELECT zip
                FROM zip_cbsa
                WHERE cbsa = '34980'
                )
                AND entity_type_code = 2.0
            ),
        referrals50 AS (
            SELECT *
            FROM referrals
            WHERE average_day_wait < 50
            AND transaction_count >= 50
        ),
        qualifying_refs AS (
            SELECT *
            FROM metro_entity_2
            INNER JOIN referral_to
            ON metro_entity_2.npi = referral_to.to_npi
            INNER JOIN referrals50
            USING (referral_id)
        ),
        entity1 AS (
            SELECT npi
            FROM profile
            INNER JOIN taxonomy
            ON profile.taxonomy_code = taxonomy.code
            WHERE entity_type_code = 1.0
        )
        SELECT 
            to_npi,
            referral_id,
            qf.entity_type_code,
            qf.taxonomy_code,
            classification,
            specialization,
            qf."provider_organization_name_(legal_business_name)" AS organization,
            qf.provider_first_line_business_practice_location_address || '
            ' || qf.provider_second_line_business_practice_location_address || '
            ' || qf.provider_business_practice_location_address_city_name || ', ' || 
            qf.provider_business_practice_location_address_state_name || ' ' || 
            qf.provider_business_practice_location_address_postal_code AS address
        FROM qualifying_refs AS qf
        INNER JOIN referral_from
        USING (referral_id)
        INNER JOIN entity1
        ON referral_from.from_npi = entity1.npi;

"""

groups = pd.read_sql(query, db)

Wall time: 24.5 s


In [4]:
%%time
# profiles of all providers
# with referrals to metro nashville groups (entity 2) having
# referrals of over 50 transactions and under 50 average wait time
query = """
        
        WITH metro_entity_2 AS (
            SELECT *
            FROM profile
            WHERE provider_business_practice_location_address_postal_code IN (
                SELECT zip
                FROM zip_cbsa
                WHERE cbsa = '34980'
                )
                AND entity_type_code = 2.0
            ),
        referrals50 AS (
            SELECT *
            FROM referrals
            WHERE average_day_wait < 50
            AND transaction_count >= 50
        ),
        qualifying_refs AS (
            SELECT 
                referral_id
            FROM metro_entity_2
            INNER JOIN referral_to
            ON metro_entity_2.npi = referral_to.to_npi
            INNER JOIN referrals50
            USING (referral_id)
        ),
        entity1 AS (
            SELECT *
            FROM profile
            INNER JOIN taxonomy
            ON profile.taxonomy_code = taxonomy.code
            WHERE entity_type_code = 1.0
        )
        SELECT
            from_npi,
            referral_id,
            entity_type_code,
            "provider_last_name_(legal_name)" || ' ' || provider_first_name AS name,
            taxonomy_code,
            classification,
            specialization,
            "provider_organization_name_(legal_business_name)" AS organization,
            provider_first_line_business_practice_location_address || '
            ' || provider_second_line_business_practice_location_address || '
            ' || provider_business_practice_location_address_city_name || ', ' || 
            provider_business_practice_location_address_state_name || ' ' || 
            provider_business_practice_location_address_postal_code AS address        
        FROM qualifying_refs
        INNER JOIN referral_from
        USING (referral_id)
        INNER JOIN entity1
        ON referral_from.from_npi = entity1.npi;

"""

providers = pd.read_sql(query, db)

Wall time: 22.8 s


In [5]:
%%time
# referrals from providers (entity 1) to metro nashville groups (entity 2) 
# having over 50 transactions and under 50 average wait time
query = """
        
        WITH metro_entity_2 AS (
            SELECT *
            FROM profile
            WHERE provider_business_practice_location_address_postal_code IN (
                SELECT zip
                FROM zip_cbsa
                WHERE cbsa = '34980'
                )
                AND entity_type_code = 2.0
            ),
        referrals50 AS (
            SELECT *
            FROM referrals
            WHERE average_day_wait < 50
            AND transaction_count >= 50
        ),
        qualifying_refs AS (
            SELECT
                to_npi,
                referral_id,
                patient_count,
                transaction_count,
                average_day_wait,
                std_day_wait  
            FROM metro_entity_2
            INNER JOIN referral_to
            ON metro_entity_2.npi = referral_to.to_npi
            INNER JOIN referrals50
            USING (referral_id)
        ),
        entity1 AS (
            SELECT *
            FROM profile
            INNER JOIN taxonomy
            ON profile.taxonomy_code = taxonomy.code
            WHERE entity_type_code = 1.0
        )
        SELECT
            from_npi,
            to_npi,
            referral_id,
            patient_count,
            transaction_count,
            average_day_wait,
            std_day_wait
        FROM qualifying_refs
        INNER JOIN referral_from
        USING (referral_id)
        INNER JOIN entity1
        ON referral_from.from_npi = entity1.npi;

"""

referrals = pd.read_sql(query, db)

Wall time: 25.3 s


In [6]:
db.close()

In [7]:
providers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69427 entries, 0 to 69426
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   from_npi          69427 non-null  int64  
 1   referral_id       69427 non-null  int64  
 2   entity_type_code  69427 non-null  float64
 3   name              69419 non-null  object 
 4   taxonomy_code     69427 non-null  object 
 5   classification    69427 non-null  object 
 6   specialization    34480 non-null  object 
 7   organization      0 non-null      object 
 8   address           26308 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 4.8+ MB


In [8]:
providers['from_npi'].nunique()

17773

In [9]:
groups['to_npi'].nunique()
groups['organization'].nunique()

1262

In [12]:
(groups
     .loc[groups['organization'].str.contains('VANDERBILT', na = False),
          ['organization', 'classification', 'specialization']]
     .drop_duplicates()
)

,organization,classification,specialization
8700,VANDERBILT HEALTH AND WILLIAMSON MEDICAL CENTE...,Internal Medicine,None
18178,VANDERBILT UNIVERSITY MEDICAL CENTER,Psychiatric Unit,None
18182,VANDERBILT UNIVERSITY MEDICAL CENTER,Rehabilitation Unit,None
18185,VANDERBILT UNIVERSITY MEDICAL CENTER,General Acute Care Hospital,None
26037,"VANDERBILT MAURY RADIATION ONCOLOGY, LLC",Clinic/Center,"Oncology, Radiation"
41143,VANDERBILT UNIVERSITY MEDICAL CENTER,Clinic/Center,End-Stage Renal Disease (ESRD) Treatment
44951,"VANDERBILT IMAGING SERVICES, LLC",Clinic/Center,Radiology
50958,VANDERBILT STALLWORTH REHABILITATION HOSPITAL LP,Rehabilitation Hospital,None
51038,"VANDERBILT IMAGING SERVICES, LLC",Radiology,Diagnostic Radiology
59038,"VANDERBILT HOME CARE SERVICES, LLC",Home Health,None


In [15]:
(groups
     .loc[groups['organization'].str.contains('HOSPITAL'), 
          ['organization', 'classification', 'specialization']]
     .drop_duplicates()
     .sort_values('classification')
)

,organization,classification,specialization
911,WYTHE COUNTY COMMUNITY HOSPITAL LLC,Anesthesiology,None
46813,METROPOLITAN HOSPITAL AUTHORITY,Clinic/Center,Multi-Specialty
24481,SHELBYVILLE HOSPITAL COMPANY LLC,Clinic/Center,Rural Health
20180,"SAINT THOMAS RIVER PARK HOSPITAL, LLC",Clinic/Center,Ambulatory Surgical
19412,MAURY REGIONAL HOSPITAL,Clinic/Center,Federally Qualified Health Center (FQHC)
8522,HOSPITAL BASED MEDICAL SERVICES OF TENNESSEE-I PC,Emergency Medicine,None
46760,NASHVILLE GENERAL HOSPITAL,General Acute Care Hospital,None
45127,HTI MEMORIAL HOSPITAL CORPORATION,General Acute Care Hospital,None
43782,SAINT THOMAS WEST HOSPITAL,General Acute Care Hospital,None
26270,"SAINT THOMAS STONES RIVER HOSPITAL, LLC",General Acute Care Hospital,None


In [16]:
(groups
     .loc[groups['classification'].str.contains('Hospital'), 
          ['organization', 'classification', 'specialization']]
     .drop_duplicates()
     .sort_values('classification')
)

,organization,classification,specialization
185,"HCA HEALTH SERVICES OF TENNESSEE, INC.",General Acute Care Hospital,Critical Access
45127,HTI MEMORIAL HOSPITAL CORPORATION,General Acute Care Hospital,None
43782,SAINT THOMAS WEST HOSPITAL,General Acute Care Hospital,None
26270,"SAINT THOMAS STONES RIVER HOSPITAL, LLC",General Acute Care Hospital,None
25460,NORTHCREST MEDICAL CENTER,General Acute Care Hospital,None
24615,"SAINT THOMAS DEKALB HOSPITAL, LLC",General Acute Care Hospital,None
24000,SHELBYVILLE HOSPITAL COMPANY LLC,General Acute Care Hospital,None
22153,SAINT THOMAS RUTHERFORD HOSPITAL,General Acute Care Hospital,None
20701,"TRUSTPOINT HOSPITAL, LLC",General Acute Care Hospital,None
66649,MAURY REGIONAL HOSPITAL,General Acute Care Hospital,None


In [ ]:
referrals